In [1]:
import _init_path

import theano.sandbox.cuda
theano.sandbox.cuda.use("gpu1")

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [7]:
import numpy as np

from datamanagement import *
from modelmanagement import *
from backpropagations import *
from activations import *

from baseModel import building_model

In [3]:
print 'load data ...',
data, vocab = load_sst(data_file='../data/pickles/sst_word_so_mincut1.pkl')
word2vec, word2idx = load_w2v(data_file='../data/pickles/sst_word2vec_mincut1.pkl')
assert len(vocab)==len(word2idx)
print '\tdone.'

load data ... 	done.


In [4]:
opts = dict()
# model 
opts['miniBatch'] = 50
# embedding
opts['vocaSize'] = len(vocab)
opts['embdSize'] = 300
opts['embdUpdate'] = False
# recurrent convolutional layer
opts['maxLen'] = max(data[0][2])
opts['maxLen_test'] = max(data[2][2])
opts['fltrRC'] = [[(30,5,5)]]
opts['borderMode'] = 'full' # one of ['valid', 'full', 'half']
opts['numStep'] = 1
opts['activationRC'] = relu # [elu, relu, tanh, iden, sigm]
opts['activationFC'] = tanh # [elu, relu, tanh, iden, sigm]
# pooling
opts['pool'] = True
opts['poolMode'] ='max' # one of ['d_k_max', 'k_max', 'max', 'spatial]
opts['kTop'] = 1
# multi-layer perceptron
opts['outUnit'] = [5]
# normalizations
opts['LRN'] = True # local response normalization
opts['dropRC'] = True
opts['dropFC'] = True
opts['dropRate'] = 0.5
# generalizations
opts['L1'] = 0 # L1-norm weight decay
opts['L2'] = 1e-4 # L2-norm weight decay
# training parameters
opts['updateRule'] = adam # one of ['adam', 'adadelta', 'adagrad', 'sgd', 'sgd_momentum']

learning_opts = dict()
# learning options
learning_opts['lr'] = 0.001 # initial learning rate
learning_opts['lrDecay'] = 0.5
learning_opts['lrDecayStep'] = 5
learning_opts['maxEpoch'] = 15

In [5]:
print 'fitting length of sentences ...',
train_data = fit_length(data[0], max_len=opts['maxLen'])
valid_data = fit_length(data[1], max_len=opts['maxLen_test'])
test_data = fit_length(data[2], max_len=opts['maxLen_test'])
print '\tdone.'

print 'fitting mini-batch size for training data ...',
train_data = fit_batch_add(train_data, opts['miniBatch'])
print '\tdone.'

fitting length of sentences ... 	done.
fitting mini-batch size for training data ... 	done.


# experiment

In [8]:
train_model, valid_model, test_model, layers, params = building_model(opts)

build layers of RCNN ... 	done.
build theano function of validation model ... 	done.
build cost function and get update rule ... 	done.
build theano function of training model ... 	done.
build testing layers of RCNN ... 	done.
build theano function of testing model ... 	done.


In [9]:
initializing_model(layers, params, svd_initialize=False, w2v_pretrained=word2vec.astype(np.float32))

ValueError: ('object of too small depth for desired array', 'Container name "W"')

In [ ]:
testing_model(test_data, test_model)

In [ ]:
fresult = 'example'
training_model([train_data, valid_data, test_data], [train_model, valid_model, test_model], opts, learning_opts, fresult, params)